In [7]:
# このセルを実行（▶️をクリック）
import subprocess
from pathlib import Path
from datetime import datetime
import json
import shutil
import time

template_path = Path("/Users/user/Library/CloudStorage/Box-Box/Personal/dev/homework/general/template.html")
with template_path.open("r", encoding="utf-8") as f:
    template = f.read()

assignment_path = Path("/Users/user/Library/CloudStorage/Box-Box/Personal/dev/homework/general/assignment.txt")
assignment_text = assignment_path.read_text(encoding="utf-8").strip()

input_path = Path("/Users/user/Library/CloudStorage/Box-Box/Personal/dev/homework/general/input.txt")
input_lines = input_path.read_text(encoding="utf-8").strip().split('\n')
subject_line = input_lines[0] if len(input_lines) > 0 else ""
section_line = input_lines[1] if len(input_lines) > 1 else ""

# 既存のsolution.htmlをarchiveに移動（最初に実行）
print('📦 既存の解答ファイルをアーカイブに移動します...')
output_dir = Path("/Users/user/Library/CloudStorage/Box-Box/Personal/dev/homework/general/output")
archive_dir = Path("/Users/user/Library/CloudStorage/Box-Box/Personal/dev/homework/general/archive")
archive_dir.mkdir(exist_ok=True)

solution_path = output_dir / "solution.html"
if solution_path.exists():
    now = datetime.now()
    timestamp = now.strftime("%Y%m%d_%H%M%S")
    archive_filename = f"solution_{assignment_text}_{timestamp}.html"
    archive_path = archive_dir / archive_filename
    try:
        shutil.move(str(solution_path), str(archive_path))
        print(f'✅ 既存の解答をアーカイブしました: {archive_filename}')
        print(f'   移動先: {archive_path}')
    except Exception as e:
        print(f'⚠️ アーカイブ処理でエラーが発生しました: {e}')
        print('   手動でアーカイブしてください。')
else:
    print('ℹ️ 既存の解答ファイルはありません。新しい解答を作成します。')

def send_prompt(prompt_text, prompt_number, total_prompts, wait_time=1.0):
    """プロンプトを送信する関数"""
    try:
        subprocess.run('pbcopy', text=True, input=prompt_text, check=True)
        if prompt_number == 1:
            # 最初のプロンプトは新しいComposerを開く
            subprocess.run(['osascript', '-e', '''
            tell application "Cursor"
                activate
                delay 0.3
            end tell
            tell application "System Events"
                keystroke "i" using command down
                delay 0.3
                keystroke "v" using command down
                delay 0.3
                keystroke return
                delay ''' + str(wait_time) + '''
            end tell
            '''], check=True)
        else:
            # 2回目以降は既に開いているComposerに追加
            subprocess.run(['osascript', '-e', '''
            tell application "Cursor"
                activate
                delay 0.3
            end tell
            tell application "System Events"
                keystroke "v" using command down
                delay 0.3
                keystroke return
                delay ''' + str(wait_time) + '''
            end tell
            '''], check=True)
        print(f'✅ プロンプト {prompt_number}/{total_prompts} を送信しました...')
        return True
    except Exception as e:
        print(f'❌ プロンプト {prompt_number} の送信でエラー: {e}')
        return False

# プロンプト1: 問題の理解と分析
prompt1 = f"""@homework/general/input.txt の講義資料を詳しく読み、@homework/general/assignment.txt に記載された問題「{assignment_text}」について理解してください。

【タスク】
1. 講義資料から該当する問題の内容を特定する
2. 問題で使用されている概念や定理を確認する
3. 解答に必要な情報を整理する

このステップでは解答は作成せず、問題の理解と分析のみを行ってください。"""

# プロンプト2: 解答の生成
prompt2 = f"""まず、@homework/general/output/solution.html が存在する場合は、そのファイルを適切な名前（solution_問番号_日時.htmlの形式）に変更し、@homework/general/archive/ に移動させてください。その後、前のステップで理解した内容を基に、@homework/general/assignment.txt に記載された問題「{assignment_text}」の解答を作成してください。

【出力要件】
- 出力先: @homework/general/output/solution.html（既存ファイルはアーカイブ済み）
- テンプレート: 下記のHTMLテンプレートをそのまま骨格として使用
- 問題タイトル・本文・数式・説明だけを差し替える
- class 名、タグ階層、CSS を一切変更しない
- MathJax はテンプレートの設定のまま利用し、数式は `$...$`（行内）と `$$...$$`（ディスプレイ）を使用

【執筆ルール】
- `h1` はタイトルとし、`h2` は「【解答】」固定で始める
- 問題文ブロックは `div.problem` 内に `【問題】` と対象行列を必ず記載する
- 各 `step` ブロックは講義と同じ流れに沿って段落で説明し、`step-title` は「（1）〜」形式で連番を振る
- 段落の語尾は「〜します。」で統一
- 数式を示したら必ず直後に文章で解説し、式だけで終わらせない
- `note`、`calculation`、`verification` の各ブロックもテンプレートのスタイルで使用
- 検証セクションには少なくとも 2 つの確認手順を段落で述べ、最後に「以上より求めた結果の正しさを確認しました。」という文で締める
- 末尾に追加のスクリプトや live-server のコードは禁止

--- HTML テンプレート（そのまま利用） ---
{template}
--- テンプレートここまで ---

以上を厳守し、初学者にも分かりやすい解答を生成してください。"""

# プロンプト3: 解答の確認と改善
prompt3 = """@homework/general/output/solution.html の内容を入念に確認し、以下の観点から改善してください。

【確認・改善ポイント】
1. 数式の正確性: 計算ミスや記号の誤りがないか
2. 説明の明確性: 各ステップの説明が初学者にも分かりやすいか
3. テンプレートの遵守: class名、タグ階層、CSSが変更されていないか
4. 検証の完全性: 検証セクションが適切に記載されているか
5. 文体の統一: 語尾が「〜します。」で統一されているか
6. MathJaxの記法: 数式が正しく記述されているか

必要に応じて修正を加え、より読みやすく正確な解答にしてください。"""

# プロンプト4: GitHubの更新
prompt4 = """解答が完成したので、GitHubを更新してください。

【タスク】
1. 変更内容を確認する
2. 適切なコミットメッセージでコミットする（general/ディレクトリのみをコミットすること）
3. 必要に応じてプッシュする

【重要】
- general/ディレクトリのみをコミットしてください
- 他のディレクトリ（body-theory/、c-pro/、cs/、platform/、research/など）はコミットに含めないでください
- 解答ファイル（@homework/general/output/solution.html）が正しく保存されていることを確認してから実行してください

コミット時は `git add general/` を使用してgeneral/ディレクトリのみをステージングしてください。"""

try:
    total_prompts = 4
    
    # プロンプト1: 問題の理解と分析
    if send_prompt(prompt1, 1, total_prompts, wait_time=0.1):
        time.sleep(2)  # AIの処理を待つ
        
        # プロンプト2: 解答の生成
        if send_prompt(prompt2, 2, total_prompts, wait_time=0.1):
            time.sleep(3)  # 解答生成の処理を待つ
            
            # プロンプト3: 解答の確認と改善
            if send_prompt(prompt3, 3, total_prompts, wait_time=0.1):
                time.sleep(2)  # 確認・改善の処理を待つ
                
                # プロンプト4: GitHubの更新
                if send_prompt(prompt4, 4, total_prompts, wait_time=0.1):
                    print(f'\n✅ すべてのプロンプト（{total_prompts}回）を送信完了 - 解答生成プロセスを開始しました')
                    print('各ステップでAIが処理を行います。完了を待ってください。')
                else:
                    print('⚠️ プロンプト4の送信に失敗しました')
            else:
                print('⚠️ プロンプト3の送信に失敗しました')
        else:
            print('⚠️ プロンプト2の送信に失敗しました')
    else:
        print('⚠️ プロンプト1の送信に失敗しました')
        
except Exception as e:
    print(f'❌ エラー: {e}')
    print('\n手動でプロンプトを送信する場合:')
    print(f'\n【プロンプト1 - 問題の理解と分析】\n{prompt1}')
    print(f'\n【プロンプト2 - 解答の生成】\n{prompt2}')
    print(f'\n【プロンプト3 - 解答の確認と改善】\n{prompt3}')
    print(f'\n【プロンプト4 - GitHubの更新】\n{prompt4}')


📦 既存の解答ファイルをアーカイブに移動します...
✅ 既存の解答をアーカイブしました: solution_問3 (１)_20251124_143914.html
   移動先: /Users/user/Library/CloudStorage/Box-Box/Personal/dev/homework/general/archive/solution_問3 (１)_20251124_143914.html
✅ プロンプト 1/4 を送信しました...
✅ プロンプト 2/4 を送信しました...
✅ プロンプト 3/4 を送信しました...
✅ プロンプト 4/4 を送信しました...

✅ すべてのプロンプト（4回）を送信完了 - 解答生成プロセスを開始しました
各ステップでAIが処理を行います。完了を待ってください。
